#Sample Examples

In [6]:
import pickle
from get_indices import find_substring_indices

data = pickle.load(open('/Users/sudhanshurai/Desktop/personal project/claim_finder/_results_with_records_gpt4.pkl', 'rb'))
record= data[4]


In [7]:
content = record.content
claim = record.instances[0]['claim']
instances = record.instances[0]['instances']['instances']
print(find_substring_indices(content.lower(), instances[0].lower()))

(12, 159)


Load the Dataframe

In [1]:
import pandas as pd

df = pd.read_csv('/Users/sudhanshurai/Desktop/personal project/Enterpret (1).csv')

The important columns are these I have selected 

In [2]:
columns  = ['Content', 'Summary', 'Reasons']
filter_condition = ~df[columns[0]].isna() & ~df[columns[1]].isna() & ~df[columns[2]].isna()
filtered_df = df[filter_condition]
filtered_df.to_csv('cleaned_enterpret.csv')

Convert the the dataframe to the record class

In [3]:
from preprocess import Record, create_records
records = create_records(filtered_df)
pickle.dump(records, open('_records.pkl', 'wb'))


Get response from the GPT3.5 turbo model

In [4]:
from get_results import get_result_gpt3, get_result_gpt4, get_result_nshot
for record in records:
	record.instances = get_result_gpt3(record)
pickle.dump(records, open('_results_with_records_gp3.pkl', 'wb'))


Select some handful examples and pass them to GPT4 model to do Few shot prompting

In [ ]:
res10 = get_result_gpt4(records[10])
res43 = get_result_gpt4(records[43])
res38 = get_result_gpt4(records[38])

res_vals = [res10, res43, res38]
idxs=[10, 43, 38]
examples='''Some Examples are these:- 
'''
for pos in range(3):
	USER_PROMPT ='''
	Claim:
	{claim}
	Conversation:
	{content}
	Response (JSON Array):
	'''
	record = records[idxs[pos]]
	res = res_vals[pos]
	eg = USER_PROMPT.format_map({'claim': res_vals[pos][0]['claim'], 'content':record.content})+'\n'+res_vals[pos][0]['raw_instances']
	examples+=eg

pickle.dump(examples, open('_examples_via_gpt4.pkl', 'wb'))

Use these prompts as examples for GPT3.5 model

In [ ]:

from copy import deepcopy
extra_records = deepcopy(records)
for record in extra_records:
	record.instances = get_result_nshot(record, examples)
pickle.dump(extra_records, open('_results_with_records_gpt4.pkl', 'wb'))


In [18]:
gpt3_results = pickle.load(open('/Users/sudhanshurai/Desktop/personal project/claim_finder/_results_with_records_gp3.pkl', 'rb'))
gpt4_results = pickle.load(open('/Users/sudhanshurai/Desktop/personal project/claim_finder/_results_with_records_gpt4.pkl', 'rb'))

Find the indices from the record

In [ ]:
from get_indices import find_substring_indices

for record in gpt3_results:
	content = record.content
	claim = record.instances[0]['claim']
	instances = record.instances[0]['instances']['instances']
	print(find_substring_indices(content.lower(), instances[0].lower()))



Use LLM to evaluate the results

In [28]:
from evaluate import *
gpt3_evaluation_res  = evaluate_all_the_records(gpt3_results)

In [29]:
avg_gpt3=0
for row in gpt3_evaluation_res:
    avg_gpt3+=(row['score'])
print(avg_gpt3/len(gpt3_evaluation_res)) 